In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [3]:
ds=pd.read_csv('D:/R/Pythonfile/spam.csv')
ds.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
ds['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [5]:
#Balancing dataset
ds_spam=ds[ds['Category']=='spam']
ds_ham=ds[ds['Category']=='ham']
ds_ham_downsampled=ds_ham.sample(ds_spam.shape[0])  # shape:输出是一元组 (行数,列数)
ds_balanced=pd.concat([ds_ham_downsampled,ds_spam]) # concat默认行(上下)拼接即axis=0
ds_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [6]:
#Adding labels
ds_balanced['spam']=ds_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
#ds_balanced['spam']=ds_balanced['Category'].map({'spam':1,'ham':0})
ds_balanced.sample(5)

,Category,Message,spam
1970,spam,You have won a guaranteed £200 award or even £...,1
1146,spam,"Thank you, winner notified by sms. Good Luck! ...",1
4330,ham,1Apple/Day=No Doctor. 1Tulsi Leaf/Day=No Cance...,0
1117,ham,Hey... What time is your driving on fri? We go...,0
3423,spam,Freemsg: 1-month unlimited free calls! Activat...,1


In [7]:
#Splitting labeled dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds_balanced['Message'],ds_balanced['spam'], stratify=ds_balanced['spam'])

In [25]:
#Downloading the Bert model
#链接因外网无法访问
#bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
#bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

#下载并解压缩后才可访问使用
bert_preprocess = hub.KerasLayer("C:/Py/Lib/site-packages/tensorflow_hub/bert_en_uncased_preprocess_3/")
bert_encoder = hub.KerasLayer("C:/Py/Lib/site-packages/tensorflow_hub/bert_en_uncased_L-12_H-768_A-12_4/")

In [26]:
#Initializing the BERT layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [29]:
#Initializing the neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_3 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [30]:
#Model compiling
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=METRICS)

In [31]:
#Fitting the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 419s 12s/step - loss: 0.6401 - accuracy: 0.6205 - precision: 0.6347 - recall: 0.5679
Epoch 2/10
35/35 [==============================] - 427s 12s/step - loss: 0.5183 - accuracy: 0.7866 - precision: 0.7744 - recall: 0.8089
Epoch 3/10
35/35 [==============================] - 409s 12s/step - loss: 0.4475 - accuracy: 0.8438 - precision: 0.8325 - recall: 0.8607
Epoch 4/10
35/35 [==============================] - 396s 11s/step - loss: 0.4148 - accuracy: 0.8554 - precision: 0.8479 - recall: 0.8661
Epoch 5/10
35/35 [==============================] - 404s 12s/step - loss: 0.3712 - accuracy: 0.8741 - precision: 0.8606 - recall: 0.8929
Epoch 6/10
35/35 [==============================] - 466s 13s/step - loss: 0.3510 - accuracy: 0.8750 - precision: 0.8671 - recall: 0.8857
Epoch 7/10
35/35 [==============================] - 427s 12s/step - loss: 0.3300 - accuracy: 0.8929 - precision: 0.8754 - recall: 0.9161
Epoch 8/10
35/35 [=======================

In [32]:
#Evaluating model using the testing dataset
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten() #The model.predict method will give the prediction results which are in a 2D array, but we want our results in a 1D array. To convert the result from the 2D to 1D array we use the y_predicted.flatten() function.

'''Since we used a sigmoid activation function, the prediction probabilities will lie between 0.0 to 1.0. 
So, if the prediction result is > 0.5 the output should be 1, 
and if it is < 0.5, the output should be 0.'''
import numpy as np
y_predicted = np.where(y_predicted >0.5, 1, 0)
y_predicted

12/12 [==============================] - 132s 11s/step


array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,

In [33]:
#Making predictions
sample_dataset = [
 'You can win a lot of money, register in the link below',
 'You have an iPhone 10, spin the image below to claim your prize and it will be delivered in your door step',
 'You have an offer, the company will give you 50% off on every item purchased.',
 "Hey Bravin, don't be late for the meeting tomorrow will start lot exactly 10:30 am",
 "See you monday, we have alot to talk about the future of this company ."
]

model.predict(sample_dataset)

1/1 [==============================] - 4s 4s/step


array([[0.67768383],
       [0.9040618 ],
       [0.52943337],
       [0.65563047],
       [0.16737969]], dtype=float32)